### 18.6. 허깅페이스

In [1]:
using PyCall
pytf = pyimport("transformers")

2023-05-22 07:31:59.934875: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 07:32:00.176498: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-22 07:32:00.211215: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-22 07:32:00.211232: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

PyObject <module 'transformers' from '/home/tyfun/.local/lib/python3.10/site-packages/transformers/__init__.py'>

In [4]:
# 파이썬에 transformers 모듈이 설치되어 있지 않은 경우
#@pyimport pip
#pip.main(["install", "transformers"])

BERT base model (uncased)

In [2]:
unmasker = pytf.pipeline("fill-mask", model="bert-base-uncased")
unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


5-element Vector{Dict{Any, Any}}:
 Dict("score" => 0.10731087625026703, "token" => 4827, "token_str" => "fashion", "sequence" => "hello i'm a fashion model.")
 Dict("score" => 0.08774468302726746, "token" => 2535, "token_str" => "role", "sequence" => "hello i'm a role model.")
 Dict("score" => 0.053383879363536835, "token" => 2047, "token_str" => "new", "sequence" => "hello i'm a new model.")
 Dict("score" => 0.046672191470861435, "token" => 3565, "token_str" => "super", "sequence" => "hello i'm a super model.")
 Dict("score" => 0.027095822617411613, "token" => 2986, "token_str" => "fine", "sequence" => "hello i'm a fine model.")

In [3]:
using CSV, DataFrames
imdb = CSV.read("/home/tyfun/Downloads/IMDB Dataset.csv", DataFrame);
text = replace.(imdb.review, "<br />" => " ")[1:5];
tokenizer = pytf.BertTokenizer.from_pretrained("bert-base-uncased");
model = pytf.BertModel.from_pretrained("bert-base-uncased");
encoded_input = tokenizer(text, return_tensors="pt", 
    padding=true, truncation=true);
output = model(
    input_ids = encoded_input.input_ids,
    token_type_ids = encoded_input.token_type_ids,
    attention_mask = encoded_input.attention_mask);
features = output["last_hidden_state"].detach().numpy()[:,1,:]'

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768×5 adjoint(::Matrix{Float32}) with eltype Float32:
  0.183457   -0.107541     -0.249168     0.0909832   -0.2782
 -0.416676    0.053653     -0.35873     -0.463723    -0.193687
 -0.308941   -0.0827718    -0.246794     0.139111     0.0527724
 -0.212909   -0.46497       0.143973     0.152972    -0.016622
 -0.472088   -0.031428     -0.318534    -0.220803     0.175561
 -0.395376   -0.460735     -0.483444    -0.600356    -0.760108
  0.174218   -0.0703339    -0.205374     0.187732     0.00556535
  0.363432    0.337521      0.511941     0.432029     0.29052
  0.458574    0.382625      0.822231     0.175565     0.429204
 -0.236621   -0.0891502    -0.290467    -0.369064    -0.890835
  0.163833    0.0246524     0.286978     0.294781     0.142459
 -0.12807     0.114612     -0.0676048    0.34227      0.311857
 -0.303468    0.129693      0.289223     0.147675     0.163047
  ⋮                                                  
 -0.210652   -0.447384     -0.268612    -0.441203    -0.229407
 -0.426637

In [5]:
import FileIO
dic = FileIO.load("imdb_bert_feat.jld2");
feat_prev = dic["X"][:,1:5]

┌ Warning: type OneHotArrays.OneHotArray{UInt32,1,2,Vector{UInt32}} does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/ryhNR/src/data/reconstructing_datatypes.jl:495


768×5 Matrix{Float32}:
  0.184805   -0.103944     -0.248619     0.0923059   -0.275131
 -0.414875    0.0555436    -0.356611    -0.461243    -0.192297
 -0.306582   -0.0797797    -0.246718     0.138624     0.0530732
 -0.214175   -0.467376      0.144437     0.151393    -0.0189914
 -0.473402   -0.0332733    -0.317315    -0.222228     0.175625
 -0.397253   -0.463901     -0.485493    -0.602911    -0.761745
  0.171968   -0.0710427    -0.208184     0.184724     0.00209193
  0.360662    0.336918      0.511018     0.432924     0.287636
  0.455813    0.381465      0.82192      0.175865     0.430091
 -0.237144   -0.0899128    -0.291191    -0.36698     -0.888839
  0.166349    0.027981      0.286802     0.295981     0.14278
 -0.125876    0.113792     -0.0680187    0.343161     0.314132
 -0.303973    0.12852       0.28675      0.145333     0.163611
  ⋮                                                  
 -0.211285   -0.446802     -0.26842     -0.441754    -0.225309
 -0.426631   -0.507301     -0.803833  

In [7]:
using Distances
colwise(cosine_dist, feat_prev, features)

5-element Vector{Float32}:
 7.8082085f-6
 4.351139f-6
 4.2915344f-6
 8.761883f-6
 7.0929527f-6

GPT-2

In [8]:
generator = pytf.pipeline("text-generation", model="gpt2")
generator("I like to write julia code", max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


5-element Vector{Dict{Any, Any}}:
 Dict("generated_text" => "I like to write julia code. The problem is that there are no Java interfaces to it, so that's why I need to do some more")
 Dict("generated_text" => "I like to write julia code to help test and generate functions I've written before. The julia tools make it easy to create and use libraries")
 Dict("generated_text" => "I like to write julia code.\n\nThe problem\n\nThe problem with Julia is that it doesn't have a single built-in")
 Dict("generated_text" => "I like to write julia code for the Python code, just replace `cat` with `sudo pythonjulia`. I just tried using the Python")
 Dict("generated_text" => "I like to write julia code where I'm not writing actual code.\n\nMy version of \"simple julia-ruby-server\" starts")

BART

In [9]:
summarizer = pytf.pipeline("summarization", model="facebook/bart-large-cnn")

PyObject <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x7f3d41233fa0>

In [10]:
ARTICLE = """ 
At least 10 people were killed and 55 were injured in the southern Ukrainian city of Kherson after Russian shelling hit the area on Saturday, Yaroslav Yanushevych, head of the Kherson region military administration, said in a new update on the death toll.
Yanushevych called on Kherson residents to donate blood to help save the lives of people injured in the shelling. 
He also said the Russians used "Grad multiple-launch rocket systems to hit Kherson city center," and 18 people, among 55, are in critical condition at the moment. 
The head of the regional administration noted there were no children among the victims of Saturday morning's shelling.
At the same time, he reported five people were killed and 17 were injured Friday as a result of Russian shelling.
"Among the injured is a six-year-old girl, doctors fought for her life and managed to save her," Yanushevych said. "But, unfortunately, the six-year-old girl lost her eye and her ear. She has a broken leg. We will evacuate her to Kyiv." 
Yanushevych said the shelling of the city continued all day Saturday, especially the areas along the Dnipro river, and it prevented the rescuers from doing their job of clearing the rubble.
The areas along the Dnipro river, such as Hydropark, Antonivka and Navtohavan have no heating or electricity. Yanushevych said people who live in these areas are under constant heavy shelling and "need to immediately evacuate." 
"It is almost impossible to restore the infrastructure in these areas," Yanushevych said.
""";

In [11]:
summarizer(ARTICLE, max_length=130, min_length=30)

1-element Vector{Dict{Any, Any}}:
 Dict("summary_text" => "At least 10 people were killed and 55 were injured in the southern Ukrainian city of Kherson. 18 people, among 55, are in critical condition at the moment. The head of the regional administration noted no children among the victims.")